In [29]:
import numpy as np
import pandas as pd
#for reading in data properly
import ast
import json

import gensim
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn import utils

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

In [30]:
all_data = pd.read_csv('train.csv')
all_data = all_data.dropna(subset=['overview', 'genres']) #drop cols without overview or genre (data we use or labels)
genre_set = {'Comedy'}

In [31]:
def text_to_list(x):
    if pd.isna(x):
        return ''
    else:
        return ast.literal_eval(x)

def parse_json(x):
    try:
        return json.loads(x.replace("'", '"'))[0]['name']
    except:
        return ''
    
def parse_all_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        for i in range(numElems):
            genre_set.add(json_genres[i]['name'])
    except:
        return ''
    
def parse_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = [0]*len(genre_dict) #20 0s
        for i in range(numElems):
            ret[genre_dict[(json_genres[i]['name'])]] = 1
        return ret
    except:
        return ''
    

def get_labels_as_strs(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = []#20 0s
        for i in range(numElems):
            ret.append(json_genres[i]['name'])
        return ret
    except:
        return ''

In [32]:
 def getAllGenres():
    full_data = pd.read_csv('train.csv')

    y = full_data['genres']
    y.apply(parse_all_genres_json)

In [33]:
getAllGenres()#populate the genre set

In [34]:
#get set to dictionary for indexing of target vectors
genre_dict = {}
index = 0
for genre in genre_set:
    genre_dict[genre] = index
    index += 1

In [35]:
def getGenresVects():
    y = all_data['genres']
    ret = y.apply(parse_genres_json)
    all_data['genres_vect'] = ret
    label_strs = y.apply(get_labels_as_strs)
    all_data['genres_labels'] = label_strs
    return ret

In [36]:
labels_vects = getGenresVects() #get label vectors for genres indexed by indexes in genre_dict

In [37]:
#put to lower case, remove punctation
def cleanText(text):
    text = re.sub(r'[^a-z A-Z0-9]', "", text) #maybe shouldn't remove punction between words here?
    text = text.lower()
    return text
all_data['cleanOverview'] = all_data['overview'].apply(cleanText)

In [38]:
#logistic regression data
lr_data = all_data[['cleanOverview', 'genres_labels', 'genres_vect', 'overview']]

In [39]:
train, test = train_test_split(lr_data, test_size=0.2, random_state=42)

CNN STUFF here

In [40]:
#get word embeddings
x = lr_data['cleanOverview'].values.tolist()

In [41]:
tok = [word_tokenize(sent) for sent in x]

In [42]:
model = Word2Vec(tok, min_count = 1, size=32)

In [43]:
model.wv.__getitem__('test')

array([ 0.05091951, -0.04784649,  0.11069489, -0.24993515,  0.20700265,
        0.06569795, -0.2701927 ,  0.41776818, -0.1219014 ,  0.42027968,
       -0.33430973,  0.01497537, -0.52383196,  0.08832876,  0.11108792,
       -0.01976492,  0.27744392,  0.4145325 ,  0.1559598 ,  0.56155735,
       -0.30898437, -0.00444493, -0.11159223,  0.3572997 , -0.14436567,
       -0.03715143, -0.06922381, -0.19020006, -0.31129828, -0.17555408,
       -0.5060845 , -0.565167  ], dtype=float32)

In [53]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
structure_test = Sequential()
#32 because word vectors were made to have length of 32
#200 as largest overview has 177 words and we'd like to include enttire thing
e = Embedding(100000, 32, input_length=200)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.add(GlobalMaxPooling1D())
structure_test.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 32)           3200000   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 199, 100)          6500      
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 100)               0         
Total params: 3,206,500
Trainable params: 3,206,500
Non-trainable params: 0
_________________________________________________________________


In [54]:
model_cnn_01 = Sequential()
e = Embedding(100000, 32, weights=[embedding_matrix], input_length=200, trainable=False)
model_cnn_01.add(e)
model_cnn_01.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_01.add(GlobalMaxPooling1D())
model_cnn_01.add(Dense(256, activation='relu'))
model_cnn_01.add(Dense(20, activation='sigmoid'))
model_cnn_01.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

NameError: name 'embedding_matrix' is not defined